In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing


In [2]:
kddCupTrain = pd.read_csv('kddCupTrain.csv',header=None)
print("Shape of kddCupTrain: ",kddCupTrain.shape)
print("There are any missing values: ", kddCupTrain.isnull().values.any())
print(kddCupTrain.head(3))

Shape of kddCupTrain:  (985262, 42)
There are any missing values:  False
   0    1     2   3    4      5   6   7   8   9   ...  32   33   34   35   36   
0   0  tcp  http  SF  215  45076   0   0   0   0  ...   0  0.0  0.0  0.0  0.0  \
1   0  tcp  http  SF  162   4528   0   0   0   0  ...   1  1.0  0.0  1.0  0.0   
2   0  tcp  http  SF  236   1228   0   0   0   0  ...   2  1.0  0.0  0.5  0.0   

    37   38   39   40       41  
0  0.0  0.0  0.0  0.0  normal.  
1  0.0  0.0  0.0  0.0  normal.  
2  0.0  0.0  0.0  0.0  normal.  

[3 rows x 42 columns]


In [3]:
kddCupTrain.iloc[:,-1].unique()

array(['normal.', 'ipsweep.'], dtype=object)

In [4]:
kddCupTrain.rename(columns={41:'Class'}, inplace=True)
kddCupTrain['Class'] = np.where(kddCupTrain['Class'] == 'normal.', 0, 1)

In [5]:
count_classes = pd.value_counts(kddCupTrain['Class'], sort = True)
print(count_classes)

Class
0    972781
1     12481
Name: count, dtype: int64


In [6]:
print(kddCupTrain.describe(percentiles=[]))

                   0             4             5              6         7   
count  985262.000000  9.852620e+05  9.852620e+05  985262.000000  985262.0  \
mean      215.078631  1.459258e+03  3.193730e+03       0.000007       0.0   
std      1343.633640  1.097984e+05  3.401613e+04       0.002665       0.0   
min         0.000000  0.000000e+00  0.000000e+00       0.000000       0.0   
50%         0.000000  2.300000e+02  4.060000e+02       0.000000       0.0   
max     58329.000000  8.958152e+07  1.173059e+07       1.000000       0.0   

                   8              9             10             11   
count  985262.000000  985262.000000  985262.000000  985262.000000  \
mean        0.000036       0.048908       0.000097       0.710185   
std         0.015897       0.926008       0.013058       0.453677   
min         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       1.000000   
max        14.000000      77.000000       4.00

In [7]:
kddCupTrain

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,Class
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,0
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,0
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,0
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985257,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,0
985258,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,0
985259,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,0
985260,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,0


In [8]:
kddCupTrain = pd.get_dummies(kddCupTrain, columns=[1,2,3])

In [9]:
kddCupTrain

,0,4,5,6,7,8,9,10,11,12,...,3_OTH,3_REJ,3_RSTO,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
0,0,215,45076,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
1,0,162,4528,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
2,0,236,1228,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
3,0,233,2032,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
4,0,239,486,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985257,0,212,2288,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
985258,0,219,236,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
985259,0,218,3610,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
985260,0,219,1234,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False


In [12]:
string_columns = kddCupTrain.select_dtypes(include=['object']).columns
kddCupTrain.drop(string_columns, axis=1, inplace=True)

# Remove constant columns
constant_columns = kddCupTrain.columns[(kddCupTrain.min() == kddCupTrain.max()) & (kddCupTrain.std() == 0)]
kddCupTrain.drop(constant_columns, axis=1, inplace=True)
kddCupTrain

,0,4,5,6,8,9,10,11,12,13,...,3_OTH,3_REJ,3_RSTO,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
0,0,215,45076,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
1,0,162,4528,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
2,0,236,1228,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
3,0,233,2032,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
4,0,239,486,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985257,0,212,2288,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
985258,0,219,236,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
985259,0,218,3610,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
985260,0,219,1234,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False


In [13]:
kddCupTrain = pd.concat([kddCupTrain, stringValues], axis=1)
kddCupTrain

,0,4,5,6,8,9,10,11,12,13,...,3_OTH,3_REJ,3_RSTO,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
0,0,215,45076,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
1,0,162,4528,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
2,0,236,1228,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
3,0,233,2032,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
4,0,239,486,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985257,0,212,2288,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
985258,0,219,236,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
985259,0,218,3610,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
985260,0,219,1234,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False


In [14]:
numeric_columns = kddCupTrain.select_dtypes(include=['number']).columns

numeric_columns = numeric_columns.drop('Class')

print(numeric_columns)

Index([ 0,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
      dtype='object')


In [15]:
numeric_columns = [str(item) for item in numeric_columns]

numeric_columns

['0',
 '4',
 '5',
 '6',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40']

In [16]:
# Remove uninformative variables
uninformative_columns = kddCupTrain.columns[(kddCupTrain.min() == kddCupTrain.max()) & (kddCupTrain.std() == 0)]
kddCupTrain.drop(uninformative_columns, axis=1, inplace=True)

# Print the number of columns in the transformed DataFrame
print(len(kddCupTrain.columns))


84


In [17]:
filtered_columns = kddCupTrain.select_dtypes(exclude=['number']).columns

print(filtered_columns)

len(kddCupTrain.columns)

Index(['1_icmp', '1_tcp', '1_udp', '2_IRC', '2_X11', '2_auth', '2_ctf',
       '2_domain', '2_domain_u', '2_eco_i', '2_ecr_i', '2_finger', '2_ftp',
       '2_ftp_data', '2_gopher', '2_http', '2_imap4', '2_link', '2_mtp',
       '2_name', '2_ntp_u', '2_other', '2_pop_3', '2_private', '2_red_i',
       '2_remote_job', '2_rje', '2_shell', '2_smtp', '2_ssh', '2_telnet',
       '2_tftp_u', '2_tim_i', '2_time', '2_urh_i', '2_urp_i', '2_whois',
       '3_OTH', '3_REJ', '3_RSTO', '3_RSTR', '3_S0', '3_S1', '3_S2', '3_S3',
       '3_SF', '3_SH'],
      dtype='object')


84

In [18]:
featuresList = [col for col in kddCupTrain if col != 'Class']
print('featuresList: ',featuresList)

kddCupTrain = kddCupTrain[featuresList + ['Class']]
print('\nkddCupTrain: \n')
kddCupTrain

featuresList:  [0, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, '1_icmp', '1_tcp', '1_udp', '2_IRC', '2_X11', '2_auth', '2_ctf', '2_domain', '2_domain_u', '2_eco_i', '2_ecr_i', '2_finger', '2_ftp', '2_ftp_data', '2_gopher', '2_http', '2_imap4', '2_link', '2_mtp', '2_name', '2_ntp_u', '2_other', '2_pop_3', '2_private', '2_red_i', '2_remote_job', '2_rje', '2_shell', '2_smtp', '2_ssh', '2_telnet', '2_tftp_u', '2_tim_i', '2_time', '2_urh_i', '2_urp_i', '2_whois', '3_OTH', '3_REJ', '3_RSTO', '3_RSTR', '3_S0', '3_S1', '3_S2', '3_S3', '3_SF', '3_SH']

kddCupTrain: 



,0,4,5,6,8,9,10,11,12,13,...,3_REJ,3_RSTO,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH,Class
0,0,215,45076,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,0
1,0,162,4528,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,0
2,0,236,1228,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,0
3,0,233,2032,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,0
4,0,239,486,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985257,0,212,2288,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,0
985258,0,219,236,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,0
985259,0,218,3610,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,0
985260,0,219,1234,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,0


In [19]:
kddCupTrain.columns = kddCupTrain.columns.astype(str)

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = preprocessing.StandardScaler()
scaler.fit(kddCupTrain[numeric_columns])

kddCupTrain[numeric_columns] = scaler.transform(kddCupTrain[numeric_columns])
kddCupTrain

,0,4,5,6,8,9,10,11,12,13,...,3_REJ,3_RSTO,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH,Class
0,-0.160072,-0.011332,1.231248,-0.002665,-0.002235,-0.052816,-0.007462,0.638815,-0.004408,-0.01751,...,False,False,False,False,False,False,False,True,False,0
1,-0.160072,-0.011815,0.039225,-0.002665,-0.002235,-0.052816,-0.007462,0.638815,-0.004408,-0.01751,...,False,False,False,False,False,False,False,True,False,0
2,-0.160072,-0.011141,-0.057788,-0.002665,-0.002235,-0.052816,-0.007462,0.638815,-0.004408,-0.01751,...,False,False,False,False,False,False,False,True,False,0
3,-0.160072,-0.011168,-0.034152,-0.002665,-0.002235,-0.052816,-0.007462,0.638815,-0.004408,-0.01751,...,False,False,False,False,False,False,False,True,False,0
4,-0.160072,-0.011114,-0.079601,-0.002665,-0.002235,-0.052816,-0.007462,0.638815,-0.004408,-0.01751,...,False,False,False,False,False,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985257,-0.160072,-0.011360,-0.026627,-0.002665,-0.002235,-0.052816,-0.007462,0.638815,-0.004408,-0.01751,...,False,False,False,False,False,False,False,True,False,0
985258,-0.160072,-0.011296,-0.086951,-0.002665,-0.002235,-0.052816,-0.007462,0.638815,-0.004408,-0.01751,...,False,False,False,False,False,False,False,True,False,0
985259,-0.160072,-0.011305,0.012237,-0.002665,-0.002235,-0.052816,-0.007462,0.638815,-0.004408,-0.01751,...,False,False,False,False,False,False,False,True,False,0
985260,-0.160072,-0.011296,-0.057612,-0.002665,-0.002235,-0.052816,-0.007462,0.638815,-0.004408,-0.01751,...,False,False,False,False,False,False,False,True,False,0


In [22]:
from sklearn.model_selection import train_test_split

X_train_split, X_test_split = train_test_split(kddCupTrain, test_size=0.2,
                                   stratify=kddCupTrain['Class'], 
                                   random_state=0)

In [23]:
y_train = X_train_split['Class']
X_train = X_train_split.drop(['Class'], axis=1)

y_test = X_test_split['Class']
X_test = X_test_split.drop(['Class'], axis=1)
print('Train: shape X',X_train.shape,', shape Y',y_train.shape)
print('Test: shape X',X_test.shape,', shape Y',y_test.shape)

Train: shape X (788209, 83) , shape Y (788209,)
Test: shape X (197053, 83) , shape Y (197053,)


In [24]:
X_trainNorm = X_train[y_train == 0]
X_trainNorm_val = X_trainNorm.values # Only values, axis labels removed. This is input for the Autoencoder
X_testNorm_val = X_test[y_test == 0].values # The validation data

In [25]:
print(y_train.shape)
print(X_train.shape)
X_trainNorm_val.shape

(788209,)
(788209, 83)


(778224, 83)

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model

ModuleNotFoundError: No module named 'google.protobuf'

In [ ]:
#Set parameters:
input_dim = X_trainNorm_val.shape[1]
layer1_dim = 14
encoder_dim = 7

In [ ]:
input_layer = Input(shape=(input_dim, ))

encoder1 = Dense(layer1_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder2 = Dense(encoder_dim, activation="relu")(encoder1)
# encoder2 = BatchNormalization()(encoder2)
# encoder2 = Dropout(0.2)(encoder2)
decoder1 = Dense(layer1_dim, activation='tanh')(encoder2)
decoder2 = Dense(input_dim, activation='linear')(decoder1)
print('input_layer: ',input_layer)
print('encoder1',encoder1)
print('encoder2',encoder2)
print('decoder1',decoder1)
print('decoder2',decoder2)

In [ ]:
autoencoder = Model(inputs=input_layer, outputs=decoder2)
autoencoder.summary()

In [ ]:
nb_epoch = 100
batch_size = 32

autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error')

checkpointer = ModelCheckpoint(filepath="autoencoder.h5", verbose=0, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0) # 'patience' number of not improving epochs
history = autoencoder.fit(X_trainNorm_val, X_trainNorm_val,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_testNorm_val, X_testNorm_val),
                    verbose=1,
                    callbacks=[checkpointer, #tensorboard, 
                               earlystopping]).history

In [ ]:
autoencoder = load_model('autoencoder.h5')

In [ ]:
testPredictions = autoencoder.predict(X_test)
X_test.shape,testPredictions.shape

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
testMSE = mean_squared_error(X_test.transpose(), testPredictions.transpose(),
                              multioutput='raw_values')
error_df = pd.DataFrame({'reconstruction_error': testMSE,'true_class': y_test})
error_df.head()

In [ ]:
from sklearn.metrics import (confusion_matrix, auc, roc_curve, cohen_kappa_score, accuracy_score)
import matplotlib.pyplot as plt

In [ ]:
fpr, tpr, thresholds = roc_curve(error_df.true_class, error_df.reconstruction_error)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
featuresList = [str(item) for item in featuresList]

In [ ]:
kddCupTest = pd.read_csv('kddCupTest.csv', header=None)
print(kddCupTest.head(3))

In [ ]:
kddCupTest.drop(columnsList, axis=1, inplace=True)
print(kddCupTest.head(3))

In [ ]:
def add_missing_dummy_columns( d, columns ):
    missing_cols = set( columns ) - set( d.columns )
    for c in missing_cols:
        d[c] = 0

In [ ]:
def fix_columns( d, columns ):  

    add_missing_dummy_columns( d, columns )

    # make sure we have all the columns we need
    assert( set( columns ) - set( d.columns ) == set())

    extra_cols = set( d.columns ) - set( columns )
    if extra_cols:
        print("extra columns:", extra_cols)

    d = d[ columns ]
    return d

In [ ]:
kddCupTest = pd.get_dummies(kddCupTest, columns = filtered_columns)

len(kddCupTest.columns)

In [ ]:
kddCupTest.columns = kddCupTest.columns.astype(str)

In [ ]:
add_missing_dummy_columns(kddCupTest,featuresList)

len(kddCupTest.columns)

In [ ]:
kddCupTest = fix_columns(kddCupTest,featuresList)

len(kddCupTest.columns)

In [ ]:
kddCupTest[numeric_columns] = scaler.transform(kddCupTest[numeric_columns])
kddCupTest

In [ ]:
finalPredictions = autoencoder.predict(kddCupTest)

In [ ]:
testMSE = mean_squared_error(kddCupTest.transpose(), finalPredictions.transpose(),
                              multioutput='raw_values')
print(testMSE)

In [ ]:
result_df = pd.DataFrame({'reconstruction_error': testMSE})
result_df.to_csv('filename.csv')